Text data types

There are two ways to store text data in pandas:

object -dtype NumPy array.

StringDtype extension type.

We recommend using StringDtype to store text data.

Prior to pandas 1.0, object dtype was the only option. This was unfortunate for many reasons:

You can accidentally store a mixture of strings and non-strings in an object dtype array. It’s better to have a dedicated dtype.

object dtype breaks dtype-specific operations like DataFrame.select_dtypes(). There isn’t a clear way to select just text while excluding non-text but still object-dtype columns.

When reading code, the contents of an object dtype array is less clear than 'string'.

Currently, the performance of object dtype arrays of strings and arrays.StringArray are about the same. We expect future enhancements to significantly increase the performance and lower the memory overhead of StringArray.

Before we begin, let's check out our hardware setup by running the nvidia-smi command.

In [1]:
!nvidia-smi

Fri Mar 26 20:15:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   51C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd; print('Pandas Version:', pd.__version__)
import numpy as np
import cupy as cp
import cudf; print('CuDF Version:', cudf.__version__)
import warnings
warnings.filterwarnings('ignore')


Pandas Version: 1.1.5
CuDF Version: 0.18.0


For backwards-compatibility, object dtype remains the default type we infer a list of strings to

In [3]:
# Pandas

pd.Series(["a", "b", "c"])

0    a
1    b
2    c
dtype: object

In [4]:
# cuDF

cudf.Series(["a", "b", "c"])


0    a
1    b
2    c
dtype: object

To explicitly request string dtype, specify the dtype

In [5]:
pd.Series(["a", "b", "c"], dtype="string")

0    a
1    b
2    c
dtype: string

In [6]:
cudf.Series(["a", "b", "c"], dtype="str")

0    a
1    b
2    c
dtype: object

Or astype after the Series or DataFrame is created

In [7]:
pandasSeries = pd.Series(["a", "b", "c"])
pandasSeries.astype("string")

0    a
1    b
2    c
dtype: string

In [8]:
cudfSeries = cudf.Series(["a", "b", "c"])
print(cudfSeries.astype("string"))

0    a
1    b
2    c
dtype: object


You can also use StringDtype/"string" as the dtype on non-string data and it will be converted to string dtype:

In [9]:
pandasSeries = pd.Series(["a", 2, np.nan], dtype="string")
pandasSeries
type(pandasSeries[1])

str

In [10]:
cudfSeries = cudf.Series(["a", 2, np.nan], dtype="str")
cudfSeries
type(cudfSeries[1])

str

or convert from existing pandas data:

In [11]:
pandasSeries = pd.Series([1, 2, np.nan], dtype="Int64")
pandasSeries
pandasSeries2 = pandasSeries.astype("string")
pandasSeries2
type(pandasSeries2[0])

str

In [12]:
cudfSeries1 = cudf.Series([1, 2, np.nan], dtype="int64")
cudfSeries
cudfSeries2 = cudfSeries1.astype("string")
cudfSeries2
type(cudfSeries2[0])

str

Behavior differences
These are places where the behavior of StringDtype objects differ from object dtype

1 - For StringDtype, string accessor methods that return numeric output will always return a nullable integer dtype, rather than either int or float dtype, depending on the presence of NA values. Methods returning boolean output will return a nullable boolean dtype.

In [13]:
pandasSeries = pd.Series(["a", None, "b"], dtype="string")
pandasSeries
pandasSeries.str.count("a")
pandasSeries.dropna().str.count("a")

0    1
2    0
dtype: Int64

In [14]:
cudfSeries = cudf.Series(["a", None, "b"], dtype="str")
cudfSeries
cudfSeries.str.count("a")
cudfSeries.dropna().str.count("a")

0    1
2    0
dtype: int32

Both outputs are Int64 dtype. Compare that with object-dtype

In [15]:
pandasSeries2 = pd.Series(["a", None, "b"], dtype="object")

pandasSeries2.str.count("a")

pandasSeries2.dropna().str.count("a")

0    1
2    0
dtype: int64

In [16]:
cudfSeries2 = cudf.Series(["a", None, "b"], dtype="object")

cudfSeries2.str.count("a")

cudfSeries2.dropna().str.count("a")

0    1
2    0
dtype: int32

When NA values are present, the output dtype is float64. Similarly for methods returning boolean values.

In [17]:
pandasSeries.str.isdigit()
pandasSeries.str.match("a")

0     True
1     <NA>
2    False
dtype: boolean

In [18]:
cudfSeries.str.isdigit()
cudfSeries.str.match("a")

0     True
1     <NA>
2    False
dtype: bool

Some string methods, like Series.str.decode() are not available on StringArray because StringArray only holds strings, not bytes.

In comparison operations, arrays.StringArray and Series backed by a StringArray will return an object with BooleanDtype, rather than a bool dtype object. Missing values in a StringArray will propagate in comparison operations, rather than always comparing unequal like numpy.nan.

Everything else that follows in the rest of this document applies equally to string and object dtype.


String methods

Series and Index are equipped with a set of string processing methods that make it easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the str attribute and generally have names matching the equivalent (scalar) built-in string methods:

In [19]:
pandasSeries = pd.Series(
  ....:     ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
  ....: )
  ....: 

pandasSeries.str.lower()

pandasSeries.str.upper()

pandasSeries.str.len()


0       1
1       1
2       1
3       4
4       4
5    <NA>
6       4
7       3
8       3
dtype: Int64

In [20]:
cudfSeries = cudf.Series(
  ....:     ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str"
  ....: )
  ....: 

cudfSeries.str.lower()

cudfSeries.str.upper()

cudfSeries.str.len()


0       1
1       1
2       1
3       4
4       4
5    <NA>
6       4
7       3
8       3
dtype: int32

In [21]:
pandasIdx = pd.Index([" jack", "jill ", " jesse ", "frank"])

pandasIdx.str.strip()


pandasIdx.str.lstrip()


pandasIdx.str.rstrip()


Index([' jack', 'jill', ' jesse', 'frank'], dtype='object')

In [22]:
cudfIdx = cudf.Index([" jack", "jill ", " jesse ", "frank"])

cudfIdx.str.strip()

cudfIdx.str.lstrip()

cudfIdx.str.rstrip()


StringIndex([' jack' 'jill' ' jesse' 'frank'], dtype='object')

The string methods on Index are especially useful for cleaning up or transforming DataFrame columns. For instance, you may have columns with leading or trailing whitespace:

In [23]:
pandasDataFrame = pd.DataFrame(np.random.randn(3, 2), columns=[" Column A ", " Column B "], index=range(3))
   
pandasDataFrame

,Column A,Column B
0,1.572016,-0.500262
1,1.057215,1.777395
2,0.269426,-0.387357


In [24]:
cudfDataFrame = cudf.DataFrame(np.random.randn(3, 2), columns=[" Column A ", " Column B "], index=range(3))
   
cudfDataFrame

,Column A,Column B
0,1.650230,-0.188896
1,-0.220649,-0.232021
2,-0.682399,1.576062


Since df.columns is an Index object, we can use the .str accessor

In [25]:
pandasDataFrame.columns.str.strip()

pandasDataFrame.columns.str.lower()

Index([' column a ', ' column b '], dtype='object')

In [26]:
cudfDataFrame.columns.str.strip()

cudfDataFrame.columns.str.lower()

Index([' column a ', ' column b '], dtype='object')

These string methods can then be used to clean up the columns as needed. Here we are removing leading and trailing whitespaces, lower casing all names, and replacing any remaining whitespaces with underscores:

In [27]:
pandasDataFrame.columns = pandasDataFrame.columns.str.strip().str.lower().str.replace(" ", "_")
pandasDataFrame

,column_a,column_b
0,1.572016,-0.500262
1,1.057215,1.777395
2,0.269426,-0.387357


In [28]:
cudfDataFrame.columns = cudfDataFrame.columns.str.strip().str.lower().str.replace(" ", "_")
cudfDataFrame

,column_a,column_b
0,1.650230,-0.188896
1,-0.220649,-0.232021
2,-0.682399,1.576062


Splitting and replacing strings

Methods like split return a Series of lists:

In [29]:
pandasSeries3 = pd.Series(["a_b_c", "c_d_e", np.nan, "f_g_h"], dtype="string")
pandasSeries3.str.split("_")

0    [a, b, c]
1    [c, d, e]
2         <NA>
3    [f, g, h]
dtype: object

In [30]:
cudfSeries3 = cudf.Series(["a_b_c", "c_d_e", np.nan, "f_g_h"], dtype="str")
cudfSeries3.str.split("_")

0    [a, b, c]
1    [c, d, e]
2         None
3    [f, g, h]
dtype: list

Elements in the split lists can be accessed using get or [] notation:

In [31]:
pandasSeries3.str.split("_").str.get(1)

pandasSeries3.str.split("_").str[1]

0       b
1       d
2    <NA>
3       g
dtype: object

In [33]:
cudfSeries3.str.split("_").str.get(1)

cudfSeries3.str.split("_").str[1]

RuntimeError: cuDF failure at: /opt/conda/envs/rapids/conda-bld/libcudf_1615843438163/work/cpp/src/strings/strings_column_view.cu:36: strings_column_view only supports strings

It is easy to expand this to return a DataFrame using expand.

In [34]:
pandasSeries3.str.split("_", expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


In [35]:
cudfSeries3.str.split("_", expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,<NA>,<NA>,<NA>
3,f,g,h


When original Series has StringDtype, the output columns will all be StringDtype as well.

It is also possible to limit the number of splits:

In [36]:
pandasSeries3.str.split("_", expand=True, n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


In [37]:
cudfSeries3.str.split("_", expand=True, n=1)

,0,1
0,a,b_c
1,c,d_e
2,<NA>,<NA>
3,f,g_h


rsplit is similar to split except it works in the reverse direction, i.e., from the end of the string to the beginning of the string:

In [38]:
pandasSeries.str.rsplit("_", expand=True, n=1)

,0
0,A
1,B
2,C
3,Aaba
4,Baca
5,<NA>
6,CABA
7,dog
8,cat


In [39]:
cudfSeries3.str.rsplit("_", expand=True, n=1)

,0,1
0,a_b,c
1,c_d,e
2,<NA>,<NA>
3,f_g,h


replace optionally uses regular expressions:

In [40]:
pandasSeries4 = pd.Series(
   ....:     ["A", "B", "C", "Aaba", "Baca", "", np.nan, "CABA", "dog", "cat"],
   ....:     dtype="string",
   ....: )
   ....: 

pandasSeries4 

pandasSeries4.str.replace("^.a|dog", "XX-XX ", case=False, regex=True)

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6        <NA>
7    XX-XX BA
8      XX-XX 
9     XX-XX t
dtype: string

In [41]:
cudfSeries4 = cudf.Series(
   ....:     ["A", "B", "C", "Aaba", "Baca", "", np.nan, "CABA", "dog", "cat"],
   ....:     dtype="str",
   ....: )
   ....: 

cudfSeries4  
cudfSeries4.str.replace("^.a|dog", "XX-XX ",  regex=True)
# s4.str.replace("^.a|dog", "XX-XX ",  regex=True) case=False,

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6        <NA>
7        CABA
8      XX-XX 
9     XX-XX t
dtype: object

If you want literal replacement of a string (equivalent to str.replace()), you can set the optional regex parameter to False, rather than escaping each character. In this case both pat and repl must be strings:

In [42]:
pandasdollars = pd.Series(["12", "-$10", "$10,000"], dtype="string")

# These lines are equivalent
pandasdollars.str.replace(r"-\$", "-", regex=True)
pandasdollars.str.replace("-$", "-", regex=False)

0         12
1        -10
2    $10,000
dtype: string

In [43]:
cudfDollars = cudf.Series(["12", "-$10", "$10,000"], dtype="str")

# These lines are equivalent
cudfDollars.str.replace(r"-\$", "-", regex=True)
cudfDollars.str.replace("-$", "-", regex=False)

0         12
1        -10
2    $10,000
dtype: object

The replace method can also take a callable as replacement. It is called on every pat using re.sub(). The callable should expect one positional argument (a regex object) and return a string

In [46]:
# Reverse every lowercase alphabetic word
pandasPat = r"[a-z]+"

def repl(m):
            return m.group(0)[::-1]
    

pd.Series(["foo 123", "bar baz", np.nan], dtype="string").str.replace(pandasPat, repl, regex=True)
    
# Using regex groups
pandasPat = r"(?P<one>\w+) (?P<two>\w+) (?P<three>\w+)"

def repl(m):
            return m.group("two").swapcase()
    

pd.Series(["Foo Bar Baz", np.nan], dtype="string").str.replace(pandasPat, repl, regex=True)
   

0     bAR
1    <NA>
dtype: string

In [51]:
# Reverse every lowercase alphabetic word
cudfPat = r"[a-z]+"

def repl(m):
            return m.group(0)[::-1]
    

cudf.Series(["foo 123", "bar baz", np.nan], dtype="str").str.replace(cudfPat, repl, regex=True)
    
# Using regex groups
cudfPat = r"(?P<one>\w+) (?P<two>\w+) (?P<three>\w+)"

def repl(m):
            return m.group("two").swapcase()
    
# ValueError: Cannot convert value of type function  to cudf scalar
cudf.Series(["Foo Bar Baz", np.nan], dtype="str").str.replace(cudfPat, repl, regex=True)

ValueError: Cannot convert value of type function  to cudf scalar

The replace method also accepts a compiled regular expression object from re.compile() as a pattern. All flags should be included in the compiled regular expression object.

In [53]:
import re

pandas_regex_pat = re.compile(r"^.a|dog", flags=re.IGNORECASE)

pandasSeries4.str.replace(pandas_regex_pat, "XX-XX ", regex=True)

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6        <NA>
7    XX-XX BA
8      XX-XX 
9     XX-XX t
dtype: string

In [54]:
import re

cudf_regex_pat = re.compile(r"^.a|dog", flags=re.IGNORECASE)
# TypeError: object of type 're.Pattern' has no len()
cudfSeries4.str.replace(cudf_regex_pat, "XX-XX ", regex=True)

TypeError: object of type 're.Pattern' has no len()

Including a flags argument when calling replace with a compiled regular expression object will raise a ValueError.

In [55]:
pandasSeries4.str.replace(pandas_regex_pat, 'XX-XX ', flags=re.IGNORECASE)

ValueError: case and flags cannot be set when pat is a compiled regex

---------------------------------------------------------------------------
ValueError: case and flags cannot be set when pat is a compiled regex

In [56]:
cudfSeries4.str.replace(cudf_regex_pat, 'XX-XX ', flags=re.IGNORECASE)

NotImplementedError: `flags` parameter is not yet supported

---------------------------------------------------------------------------
ValueError: case and flags cannot be set when pat is a compiled regex

Concatenation


There are several ways to concatenate a Series or Index, either with itself or others, all based on cat(), resp. Index.str.cat.

Concatenating a single Series into a string

The content of a Series (or Index) can be concatenated:

In [59]:
pandasSeries = pd.Series(["a", "b", "c", "d"], dtype="string")

pandasSeries.str.cat(sep=",")

'a,b,c,d'

In [60]:
cudfSeries = cudf.Series(["a", "b", "c", "d"], dtype="str")

cudfSeries.str.cat(sep=",")

'a,b,c,d'

If not specified, the keyword sep for the separator defaults to the empty string, sep='':

In [61]:
pandasSeries.str.cat()

'abcd'

In [62]:
cudfSeries.str.cat()

'abcd'

By default, missing values are ignored. Using na_rep, they can be given a representation:

In [63]:
pandasSeriesB = pd.Series(["a", "b", np.nan, "d"], dtype="string")

pandasSeriesB.str.cat(sep=",")
pandasSeriesB.str.cat(sep=",", na_rep="-")

'a,b,-,d'

In [64]:
cudfSeriesB = cudf.Series(["a", "b", np.nan, "d"], dtype="str")

cudfSeriesB.str.cat(sep=",")
cudfSeriesB.str.cat(sep=",", na_rep="-")

'a,b,-,d'

Concatenating a Series and something list-like into a Series

The first argument to cat() can be a list-like object, provided that it matches the length of the calling Series (or Index).

In [65]:
pandasSeries.str.cat(["A", "B", "C", "D"])

0    aA
1    bB
2    cC
3    dD
dtype: string

In [66]:
cudfSeries.str.cat(["A", "B", "C", "D"])

0    aA
1    bB
2    cC
3    dD
dtype: object

Missing values on either side will result in missing values in the result as well, unless na_rep is specified:

In [67]:
pandasSeries.str.cat(pandasSeriesB)
pandasSeries.str.cat(pandasSeriesB, na_rep="-")

0    aa
1    bb
2    c-
3    dd
dtype: string

In [68]:
cudfSeries.str.cat(cudfSeriesB)
cudfSeries.str.cat(cudfSeriesB, na_rep="-")

0    aa
1    bb
2    c-
3    dd
dtype: object

Concatenating a Series and something array-like into a Series

The parameter others can also be two-dimensional. In this case, the number or rows must match the lengths of the calling Series (or Index).

In [ ]:
cudfArray = cudf.concat([cudfSeriesB, cudfSeries], axis=1)
cudfSeries
cudfArray
cudfSeries.str.cat(cudfArray, na_rep="-")

Concatenating a Series and an indexed object into a Series, with alignment

For concatenation with a Series or DataFrame, it is possible to align the indexes before concatenation by setting the join-keyword.

In [ ]:
pandasSeriesC = pd.Series(["b", "d", "a", "c"], index=[1, 3, 0, 2], dtype="string")

pandasSeries

pandasSeriesC

pandasSeries.str.cat(pandasSeriesC)
pandasSeries.str.cat(pandasSeriesC, join="left")

In [ ]:
cudfSeriesC = pd.Series(["b", "d", "a", "c"], index=[1, 3, 0, 2], dtype="string")

cudfSeries

cudfSeriesC

cudfSeries.str.cat(cudfSeriesC)
cudfSeries.str.cat(cudfSeriesC, join="left")

The usual options are available for join (one of 'left', 'outer', 'inner', 'right'). In particular, alignment also means that the different lengths do not need to coincide anymore.

In [ ]:
pandasSeriesD = pd.Series(["z", "a", "b", "d", "e"], index=[-1, 0, 1, 3, 4], dtype="string")

pandasSeries
pandasSeriesD

pandasSeries.str.cat(pandasSeriesD, join="left", na_rep="-")
pandasSeries.str.cat(pandasSeriesD, join="outer", na_rep="-")

In [ ]:
cudfSeriesD = pd.Series(["z", "a", "b", "d", "e"], index=[-1, 0, 1, 3, 4], dtype="string")

cudfSeries
cudfSeriesD

cudfSeries.str.cat(cudfSeriesD, join="left", na_rep="-")
cudfSeries.str.cat(cudfSeriesD, join="outer", na_rep="-")

The same alignment can be used when others is a DataFrame:

In [ ]:
pandasDataFrame2 = d.loc[[3, 2, 1, 0], :]

pandasSeries
pandasDataFrame2 

pandasSeries.str.cat(pandasDataFrame2 , join="left", na_rep="-")

In [ ]:
cudfDataFrame2 = d1.loc[[3, 2, 1, 0], :]

cudfSeries
cudfDataFrame2

cudfSeries.str.cat(cudfDataFrame2, join="left", na_rep="-")

Concatenating a Series and many objects into a Series

Several array-like items (specifically: Series, Index, and 1-dimensional variants of np.ndarray) can be combined in a list-like container (including iterators, dict-views, etc.).

In [ ]:
pandasSeries
pandasSeriesC 
pandasSeries.str.cat([pandasSeries, pandasSeriesC.to_numpy()], join="left")

In [ ]:
cudfSeries
cudfSeriesC = cudfSeriesC.values
# pandasSeriesC = cp.asarray(pandasSeriesC) .to_array
cudfSeries.str.cat([cudfSeriesC, cudfSeriesC], join="left")

All elements without an index (e.g. np.ndarray) within the passed list-like must match in length to the calling Series (or Index), but Series and Index may have arbitrary length (as long as alignment is not disabled with join=None):

In [ ]:
pandasSeriesD
pandasSeries.str.cat([pandasSeriesD, pandasSeriesC, pandasSeriesC.to_numpy()], join="outer", na_rep="-")

In [ ]:
cudfSeriesD
cudfSeries.str.cat([cudfSeriesD, cudfSeriesC, cudfSeriesC.], join="outer", na_rep="-")

If using join='right' on a list-like of others that contains different indexes, the union of these indexes will be used as the basis for the final concatenation:

In [ ]:
pandasSeriesC.loc[[3]]

pandasSeriesD.loc[[-1, 0]]

pandasSeries.str.cat([pandasSeriesC.loc[[3]], pandasSeriesD.loc[[-1, 0]]], join="right", na_rep="-")

In [ ]:
cudfSeriesC.loc[[3]]

cudfSeriesD.loc[[-1, 0]]

cudfSeries.str.cat([cudfSeriesC.loc[[3]], cudfSeriesD.loc[[-1, 0]]], join="right", na_rep="-")

Indexing with .str

You can use [] notation to directly index by position locations. If you index past the end of the string, the result will be a NaN.

In [ ]:
pandasSeries = pd.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string")
   

pandasSeries.str[0]
pandasSeries.str[1]

In [ ]:
cudfSeries = cudf.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str")
   

cudfSeries.str[0]
cudfSeries.str[1]

Extracting substrings

Extract first match in each subject (extract)

The extract method accepts a regular expression with at least one capture group.

Extracting a regular expression with more than one group returns a DataFrame with one column per group.

In [ ]:
pd.Series(["a1", "b2", "c3"],dtype="string",).str.extract(r"([ab])(\d)", expand=False)
    

In [ ]:
cudf.Series(["a1", "b2", "c3"],dtype="str",).str.extract(r"([ab])(\d)", expand=False)
    

Elements that do not match return a row filled with NaN. Thus, a Series of messy strings can be “converted” into a like-indexed Series or DataFrame of cleaned-up or more useful strings, without necessitating get() to access tuples or re.match objects. The dtype of the result is always object, even if no match is found and the result only contains NaN.

Named groups like

In [ ]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"(?P<letter>[ab])(?P<digit>\d)", expand=False)
    

In [ ]:
cudf.Series(["a1", "b2", "c3"], dtype="str").str.extract(r"(?P<letter>[ab])(?P<digit>\d)", expand=False)

and optional groups like

In [ ]:
pd.Series(["a1", "b2", "3"], dtype="string",
         ).str.extract(r"([ab])?(\d)", expand=False)
   

In [ ]:
cudf.Series(["a1", "b2", "3"], dtype="str",
         ).str.extract(r"([ab])?(\d)", expand=False) 

can also be used. Note that any capture group names in the regular expression will be used for column names; otherwise capture group numbers will be used.

Extracting a regular expression with one group returns a DataFrame with one column if expand=True.

In [ ]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=True)

In [ ]:
cudf.Series(["a1", "b2", "c3"], dtype="str").str.extract(r"[ab](\d)", expand=True)

It returns a Series if expand=False.

In [ ]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(r"[ab](\d)", expand=False)

In [ ]:
cudf.Series(["a1", "b2", "c3"], dtype="str").str.extract(r"[ab](\d)", expand=False)

Calling on an Index with a regex with exactly one capture group returns a DataFrame with one column if expand=True.

In [ ]:
pandasSeries = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="string")
pandasSeries
pandasSeries.index.str.extract("(?P<letter>[a-zA-Z])", expand=True)

In [ ]:
cudfSeries = cudf.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"], dtype="str")
cudfSeries
cudfSeries.index.str.extract("(?P<letter>[a-zA-Z])", expand=True)

It returns an Index if expand=False.

In [ ]:
pandasSeries.index.str.extract("(?P<letter>[a-zA-Z])", expand=False)

In [ ]:
cudfSeries.index.str.extract("(?P<letter>[a-zA-Z])", expand=False)

Calling on an Index with a regex with more than one capture group returns a DataFrame if expand=True.

In [ ]:
pandasSeries.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=True)

In [ ]:
cudfSeries.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=True)

It raises ValueError if expand=False.

In [ ]:
pandasSeries.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=False)
# ValueError: only one regex group is supported with Index

In [ ]:
cudfSeries.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=False)
# ValueError: only one regex group is supported with Index

The table below summarizes the behavior of extract(expand=False) (input subject in first column, number of groups in regex in first row)

____________________________
1 group  >1 group
____________________________
Index    Index    ValueError
____________________________
Series   Series   DataFrame

Extract all matches in each subject (extractall)

Unlike extract (which returns only the first match),

In [ ]:
pandasSeries = pd.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"], dtype="string")
pandasSeries
pandas_two_groups = "(?P<letter>[a-z])(?P<digit>[0-9])"

pandasSeries.str.extract(pandas_two_groups, expand=True)

In [ ]:
pandasSeries = cudf.Series(["a1a2", "b1", "c1"], index=["A", "B", "C"], dtype="str")
cudfSeries
cudf_two_groups = "(?P<letter>[a-z])(?P<digit>[0-9])"

cudfSeries.str.extract(cudf_two_groups, expand=True)

the extractall method returns every match. The result of extractall is always a DataFrame with a MultiIndex on its rows. The last level of the MultiIndex is named match and indicates the order in the subject.

In [ ]:
pandasSeries.str.extractall(pandas_two_groups)

In [ ]:
cudfSeries.str.extractall(cudf_two_groups)

When each subject string in the Series has exactly one match,

In [ ]:
pandasSeries = pd.Series(["a3", "b3", "c2"], dtype="string")

In [ ]:
cudfSeries = cudf.Series(["a3", "b3", "c2"], dtype="str")

then extractall(pat).xs(0, level='match') gives the same result as extract(pat).

In [ ]:
extract_result = s.str.extract(pandas_two_groups, expand=True)

extractall_result = s.str.extractall(pandas_two_groups)

extractall_result

extractall_result.xs(0, level="match")

In [ ]:
extract_result = cudfSeries.str.extract(cudf_two_groups, expand=True)

extractall_result = cudfSeries.str.extractall(cudf_two_groups)

extractall_result

extractall_result.xs(0, level="match")

Index also supports .str.extractall. It returns a DataFrame which has the same result as a Series.str.extractall with a default index (starts from 0).

In [ ]:
pandasSeries.Index(["a1a2", "b1", "c1"]).str.extractall(pandas_two_groups)

pandasSeries.Series(["a1a2", "b1", "c1"], dtype="string").str.extractall(pandas_two_groups)

In [ ]:
cudf.Index(["a1a2", "b1", "c1"]).str.extractall(cudf_two_groups)

cudf.Series(["a1a2", "b1", "c1"], dtype="str").str.extractall(cudfSeries_two_groups)

Testing for strings that match or contain a pattern

You can check whether elements contain a pattern:

In [ ]:
pattern = r"[0-9][a-z]"

pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="string",
         ).str.contains(pattern)
   

In [ ]:
pattern = r"[0-9][a-z]"

cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.contains(pattern)
   

Or whether elements match a pattern:

In [ ]:
pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="string",
         ).str.match(pattern)
   

In [ ]:
cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.match(pattern) 

New in version 1.1.0.

In [ ]:
pd.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="string",
         ).str.fullmatch(pattern)
    

In [ ]:
cudf.Series(["1", "2", "3a", "3b", "03c", "4dx"],dtype="str",
         ).str.match(pattern)

Methods like match, fullmatch, contains, startswith, and endswith take an extra na argument so missing values can be considered True or False:

In [ ]:
pandasSeries5 = pd.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string")   

pandasSeries5.str.contains("A", na=False)

In [ ]:
cudfSeries5 = cudf.Series(["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="str")   

cudfSeries5.str.contains("A", na=False)

Creating indicator variables

You can extract dummy variables from string columns. For example if they are separated by a '|':

In [ ]:
pandasSeries = pd.Series(["a", "a|b", np.nan, "a|c"], dtype="string")

pandasSeries.str.get_dummies(sep="|")

In [ ]:
cudfSeries = cudf.Series(["a", "a|b", np.nan, "a|c"], dtype="str")

cudfSeries.str.get_dummies(sep="|")

String Index also supports get_dummies which returns a MultiIndex.

In [ ]:
pandasIdx = pd.Index(["a", "a|b", np.nan, "a|c"])

pandasIdx.str.get_dummies(sep="|")

In [ ]:
cudfIdx = cudf.Index(["a", "a|b", np.nan, "a|c"])

cudfIdx.str.get_dummies(sep="|")